<a href="https://colab.research.google.com/github/KevinYih/BigDataDemo/blob/main/Frequent_Itemsets_by_kevin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Frequent Pattern Mining in Spark
source: https://colab.research.google.com/drive/1Db3fj_3jlDXgdrFWH6fUe03QF6ZT5EL5?usp=sharing

### Setup

Let's set up Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=95c1ef858f230b9f7a0eb405aad5b874a6832c813a5d816751c5a8063235ca69
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 39.6 MB of archives.
After this operation, 144 MB of additional disk space will be used.
Selecting previously unselected package

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [3]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [6]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [7]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [8]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [9]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer).

In [10]:
# YOUR CODE HERE

file_path = "order_products__train.csv"
orders = spark.read.csv(file_path, header=True, inferSchema=True)
orders.show(5)

file_path = "products.csv"
products = spark.read.csv(file_path, header=True, inferSchema=True)
products.show(5)

# JOIN products and orders ON product_id
joined_df = orders.join(products, on="product_id")

# GROUP BY order_id
grouped_df = joined_df.groupBy("order_id").agg(
    collect_list("product_name").alias("product_names")
)

grouped_df.show(truncate=False)

+--------+----------+-----------------+---------+
|order_id|product_id|add_to_cart_order|reordered|
+--------+----------+-----------------+---------+
|       1|     49302|                1|        1|
|       1|     11109|                2|        1|
|       1|     10246|                3|        0|
|       1|     49683|                4|        0|
|       1|     43633|                5|        1|
+--------+----------+-----------------+---------+
only showing top 5 rows

+----------+--------------------+--------+-------------+
|product_id|        product_name|aisle_id|department_id|
+----------+--------------------+--------+-------------+
|         1|Chocolate Sandwic...|      61|           19|
|         2|    All-Seasons Salt|     104|           13|
|         3|Robust Golden Uns...|      94|            7|
|         4|Smart Ones Classi...|      38|            1|
|         5|Green Chile Anyti...|       5|           13|
+----------+--------------------+--------+-------------+
only showing

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with

```minSupport=0.01``` and ```minConfidence=0.5```



In [12]:
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="product_names", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(grouped_df)

# Display frequent itemsets.
model.freqItemsets.show(10)

# Display generated association rules.
model.associationRules.show()

# transform examines the input items against all the association rules and summarize the
# consequents as prediction
model.transform(grouped_df).show(10)

+--------------------+----+
|               items|freq|
+--------------------+----+
|      [Green Onions]|1445|
|   [Red Raspberries]|1493|
|    [Organic Banana]|2332|
|  [Jalapeno Peppers]|1899|
|[Organic Large Ex...|2891|
|[Organic Whole St...|1993|
|[Organic Peeled W...|2460|
|             [Limes]|6033|
|[Limes, Large Lemon]|1595|
|     [Limes, Banana]|1331|
+--------------------+----+
only showing top 10 rows

+----------+----------+----------+----+-------+
|antecedent|consequent|confidence|lift|support|
+----------+----------+----------+----+-------+
+----------+----------+----------+----+-------+

+--------+--------------------+----------+
|order_id|       product_names|prediction|
+--------+--------------------+----------+
|       1|[Bulgarian Yogurt...|        []|
|      96|[Roasted Turkey, ...|        []|
|     112|[Fresh Cauliflowe...|        []|
|     218|[Natural Artisan ...|        []|
|     456|[Chorizo Pork, Pe...|        []|
|     473|[Organic Whole Mi...|        []|
| 

Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [13]:
# YOUR CODE HERE

# Compute how many frequent itemsets
num_freq_itemsets = model.freqItemsets.count()
print(f"Number of frequent itemsets: {num_freq_itemsets}")

# Compute how many association rules
num_association_rules = model.associationRules.count()
print(f"Number of association rules: {num_association_rules}")


Number of frequent itemsets: 120
Number of association rules: 0


Now retrain the FP-growth model changing only
```minsupport=0.001```
and compute how many frequent itemsets and association rules were generated.


In [14]:
# YOUR CODE HERE

fpGrowth = FPGrowth(itemsCol="product_names", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(grouped_df)

# Compute how many frequent itemsets
num_freq_itemsets = model.freqItemsets.count()
print(f"Number of frequent itemsets: {num_freq_itemsets}")

# Compute how many association rules
num_association_rules = model.associationRules.count()
print(f"Number of association rules: {num_association_rules}")

# show frequent itemsets and association rules
model.freqItemsets.show(10)
model.associationRules.show()

Number of frequent itemsets: 4444
Number of association rules: 11
+--------------------+----+
|               items|freq|
+--------------------+----+
|[White Cheddar Po...| 370|
|[Organic YoKids V...| 259|
|[Total 0% Nonfat ...| 993|
|[Total 0% Nonfat ...| 137|
|[Total 0% Nonfat ...| 157|
|[Total 0% Nonfat ...| 143|
|[Total 0% Nonfat ...| 258|
|[Organic Large Gr...| 730|
|[Organic Large Gr...| 185|
|[Organic Large Gr...| 263|
+--------------------+----+
only showing top 10 rows

+--------------------+--------------------+------------------+------------------+--------------------+
|          antecedent|          consequent|        confidence|              lift|             support|
+--------------------+--------------------+------------------+------------------+--------------------+
|[Organic Whole St...|[Bag of Organic B...|0.5314685314685315| 4.504745125675359|0.001158457118033...|
|[Organic Broccoli...|[Bag of Organic B...|0.5048231511254019| 4.278897986822536|0.001196564260073623|
|